In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import astropy.coordinates as coord
import astropy.units as u
import sys
from astropy.table import Table

if './SelfCalGroupFinder/py/' not in sys.path:
    sys.path.append('./SelfCalGroupFinder/py/')
from pyutils import *
import plotting as pp
from dataloc import *
import catalog_definitions as cat
from groupcatalog import deserialize, serialize, GroupCatalog, SDSSGroupCatalog, mstar_vmax_weighted, add_halo_columns
import groupcatalog as gc

%load_ext autoreload
%autoreload 2

In [3]:
blue_color = GLOBAL_RED_COLOR_CUT - 0.1
red_color = GLOBAL_RED_COLOR_CUT + 0.1

blue_dn = -1
red_dn = 3

results = is_quiescent_BGS_smart(np.array([7,8,9]), np.array([red_dn, np.nan, blue_dn]), np.array([blue_color, blue_color, red_color]))
assert results[0] == True
assert results[1] == False
assert results[2] == False

In [ ]:
# Basic test of multiple versions of SimpleRedshiftGuesser
# Ensure it handles arrays of inputs and gives a reasonable answer for a couple obvious cases

# Target (lost galaxies) properties
t_app_mag = np.array([19.0,18.0,12.0])
t_pobs = np.array([0.5, 0.5, 0.5])
t_q = np.array([True, True, False])

# Neighbor properties
nn_z = np.array([0.1, 0.2, 0.3])
nn_dist = np.array([250.0, 3.0, 30.0])
nn_q = np.array([True, True, False])

simple = SimpleRedshiftGuesser(None, None, ver='5.0')
z, nn_used = simple.choose_redshift(nn_z, nn_dist, t_pobs, t_app_mag, t_q, nn_q)

assert not nn_used[0]
assert nn_used[1]
assert not nn_used[2]
assert z[0] > 0.0
assert z[1] == 0.2
assert z[2] < 0.1

simple = SimpleRedshiftGuesser(None, None, ver='4.0')
z, nn_used = simple.choose_redshift(nn_z, nn_dist, t_pobs, t_app_mag, t_q, nn_q)

assert not nn_used[0]
assert nn_used[1]
assert not nn_used[2]
assert z[0] > 0.0
assert z[1] == 0.2
assert z[2] < 0.1

simple = SimpleRedshiftGuesser(None, None, ver='2.0')
z, nn_used = simple.choose_redshift(nn_z, nn_dist, t_pobs, t_app_mag, t_q, nn_q)

assert not nn_used[0]
assert nn_used[1]
assert not nn_used[2]
assert z[0] > 0.0
assert z[1] == 0.2
assert z[2] < 0.1


In [ ]:
# Basic tests for PhotometricRedshiftGuesser

# Target (lost galaxies) properties
t_app_mag = np.array([19.0,18.0,12.0])
t_pobs = np.array([0.5, 0.5, 0.5])
t_q = np.array([True, True, False])

# Neighbor properties
nn_z = np.array([[0.1, 0.2, 0.3]])
nn_dist = np.array([[250.0, 3.0, 30.0]])
nn_q = np.array([[True, True, False]])

scorer = PhotometricRedshiftGuesser()
z, nn_used = scorer.choose_redshift(nn_z, nn_dist, t_pobs, t_app_mag, t_q, nn_q)

assert np.isnan(nn_used[0])
assert nn_used[1]
assert np.isnan(nn_used[2])
assert z[0] > 0.0
assert z[1] == 0.2
assert z[2] < 0.1